# Generatieve netwerken

Recurrent Neural Networks (RNNs) en hun varianten met gated cellen, zoals Long Short Term Memory Cells (LSTMs) en Gated Recurrent Units (GRUs), bieden een mechanisme voor taalmodellering, oftewel ze kunnen de volgorde van woorden leren en voorspellingen doen voor het volgende woord in een reeks. Dit stelt ons in staat om RNNs te gebruiken voor **generatieve taken**, zoals gewone tekstgeneratie, machinevertaling en zelfs beeldbeschrijving.

In de RNN-architectuur die we in de vorige eenheid hebben besproken, produceerde elke RNN-eenheid de volgende verborgen toestand als output. We kunnen echter ook een extra output toevoegen aan elke recurrente eenheid, waardoor we een **reeks** kunnen genereren (die even lang is als de oorspronkelijke reeks). Bovendien kunnen we RNN-eenheden gebruiken die bij elke stap geen invoer accepteren, maar alleen een initiële toestandsvector nemen en vervolgens een reeks outputs produceren.

In dit notebook richten we ons op eenvoudige generatieve modellen die ons helpen tekst te genereren. Voor de eenvoud bouwen we een **karakter-niveau netwerk**, dat tekst letter voor letter genereert. Tijdens de training moeten we een tekstcorpus nemen en deze splitsen in letterreeksen.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Het opbouwen van een karaktervocabulaire

Om een generatief netwerk op karakter-niveau te bouwen, moeten we tekst splitsen in individuele karakters in plaats van woorden. De `TextVectorization`-laag die we eerder hebben gebruikt, kan dit niet doen, dus we hebben twee opties:

* Tekst handmatig laden en zelf tokeniseren, zoals in [dit officiële Keras-voorbeeld](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* De `Tokenizer`-klasse gebruiken voor tokenisatie op karakter-niveau.

We kiezen voor de tweede optie. Met `Tokenizer` kun je ook tokeniseren op woordniveau, dus het zou vrij eenvoudig moeten zijn om te schakelen tussen tokenisatie op karakter- en woordniveau.

Voor tokenisatie op karakter-niveau moeten we de parameter `char_level=True` doorgeven:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

We willen ook een speciale token gebruiken om **einde van de reeks** aan te duiden, die we `<eos>` zullen noemen. Laten we deze handmatig toevoegen aan de vocabulaire:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Een generatieve RNN trainen om titels te genereren

De manier waarop we een RNN zullen trainen om nieuwstitels te genereren is als volgt. Bij elke stap nemen we één titel, die wordt ingevoerd in een RNN, en voor elk invoerkarakter vragen we het netwerk om het volgende uitvoerkarakter te genereren:

![Afbeelding die een voorbeeld toont van RNN-generatie van het woord 'HELLO'.](../../../../../translated_images/nl/rnn-generate.56c54afb52f9781d.webp)

Voor het laatste karakter van onze reeks vragen we het netwerk om een `<eos>`-token te genereren.

Het belangrijkste verschil tussen de generatieve RNN die we hier gebruiken is dat we een uitvoer nemen van elke stap van de RNN, en niet alleen van de laatste cel. Dit kan worden bereikt door de parameter `return_sequences` op te geven aan de RNN-cel.

Dus, tijdens de training zou een invoer voor het netwerk een reeks van gecodeerde karakters van een bepaalde lengte zijn, en een uitvoer zou een reeks van dezelfde lengte zijn, maar verschoven met één element en beëindigd met `<eos>`. Een minibatch zal bestaan uit meerdere van dergelijke reeksen, en we moeten **padding** gebruiken om alle reeksen uit te lijnen.

Laten we functies maken die de dataset voor ons transformeren. Omdat we reeksen willen opvullen op minibatch-niveau, zullen we eerst de dataset groeperen door `.batch()` aan te roepen, en vervolgens `map` gebruiken om de transformatie uit te voeren. Dus, de transformatiefunctie zal een hele minibatch als parameter nemen:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Een paar belangrijke dingen die we hier doen:
* We halen eerst de daadwerkelijke tekst uit de string-tensor
* `text_to_sequences` zet de lijst van strings om in een lijst van integer-tensors
* `pad_sequences` vult die tensors aan tot hun maximale lengte
* We coderen uiteindelijk alle karakters één-op-één, en doen ook de verschuiving en `<eos>` toevoeging. We zullen binnenkort zien waarom we één-op-één gecodeerde karakters nodig hebben

Echter, deze functie is **Pythonic**, wat betekent dat deze niet automatisch kan worden vertaald naar een Tensorflow computationele grafiek. We krijgen fouten als we proberen deze functie direct in de `Dataset.map` functie te gebruiken. We moeten deze Pythonic aanroep insluiten door gebruik te maken van de `py_function` wrapper:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Het onderscheiden van Pythonic en Tensorflow transformatiefuncties kan een beetje te complex lijken, en je vraagt je misschien af waarom we de dataset niet transformeren met standaard Python-functies voordat we deze doorgeven aan `fit`. Hoewel dit zeker mogelijk is, heeft het gebruik van `Dataset.map` een groot voordeel, omdat de datatransformatie-pijplijn wordt uitgevoerd met behulp van het Tensorflow computationele grafiek, wat gebruik maakt van GPU-berekeningen en de noodzaak om gegevens tussen CPU/GPU te verplaatsen minimaliseert.

Nu kunnen we ons generatornetwerk bouwen en beginnen met trainen. Het kan gebaseerd zijn op elke recurrente cel die we in de vorige eenheid hebben besproken (simpel, LSTM of GRU). In ons voorbeeld zullen we LSTM gebruiken.

Omdat het netwerk tekens als invoer neemt en de vocabulairegrootte vrij klein is, hebben we geen embeddinglaag nodig; een one-hot-gecodeerde invoer kan direct naar de LSTM-cel gaan. De uitvoerlaag zou een `Dense` classifier zijn die de LSTM-uitvoer omzet in one-hot-gecodeerde tokennummers.

Daarnaast, omdat we werken met sequenties van variabele lengte, kunnen we een `Masking`-laag gebruiken om een masker te creëren dat het opgevulde deel van de string negeert. Dit is niet strikt noodzakelijk, omdat we niet erg geïnteresseerd zijn in alles wat voorbij het `<eos>`-token gaat, maar we zullen het gebruiken om wat ervaring op te doen met dit type laag. `input_shape` zou `(None, vocab_size)` zijn, waarbij `None` de sequentie van variabele lengte aangeeft, en de uitvoervorm is ook `(None, vocab_size)`, zoals je kunt zien in de `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Output genereren

Nu we het model hebben getraind, willen we het gebruiken om output te genereren. Allereerst hebben we een manier nodig om tekst te decoderen die wordt weergegeven door een reeks tokennummers. Hiervoor zouden we de functie `tokenizer.sequences_to_texts` kunnen gebruiken; echter, deze werkt niet goed met tokenisatie op het niveau van individuele tekens. Daarom nemen we een woordenboek van tokens uit de tokenizer (genaamd `word_index`), bouwen we een omgekeerde map en schrijven we onze eigen decodeerfunctie:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Nu gaan we genereren. We beginnen met een string `start`, coderen deze in een reeks `inp`, en vervolgens roepen we bij elke stap ons netwerk aan om het volgende teken te voorspellen.

De uitvoer van het netwerk `out` is een vector van `vocab_size` elementen die de waarschijnlijkheden van elk token vertegenwoordigen. We kunnen het meest waarschijnlijke tokennummer vinden door `argmax` te gebruiken. Vervolgens voegen we dit teken toe aan de gegenereerde lijst van tokens en gaan verder met genereren. Dit proces van het genereren van één teken wordt `size` keer herhaald om het gewenste aantal tekens te genereren, en we stoppen voortijdig wanneer `eos_token` wordt aangetroffen.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Output bemonsteren tijdens training

Omdat we geen bruikbare metrics hebben zoals *nauwkeurigheid*, is de enige manier waarop we kunnen zien dat ons model beter wordt door **bemonstering** van gegenereerde strings tijdens de training. Om dit te doen, zullen we **callbacks** gebruiken, oftewel functies die we kunnen doorgeven aan de `fit`-functie en die periodiek tijdens de training worden aangeroepen.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Dit voorbeeld genereert al behoorlijk goede tekst, maar er zijn verschillende manieren om het verder te verbeteren:

* **Meer tekst**. We hebben alleen titels gebruikt voor onze taak, maar je kunt experimenteren met volledige tekst. Onthoud dat RNN's niet zo goed zijn in het verwerken van lange reeksen, dus het is logisch om ze op te splitsen in kortere zinnen, of altijd te trainen op een vaste sequentielengte van een vooraf gedefinieerde waarde `num_chars` (bijvoorbeeld 256). Je kunt proberen het bovenstaande voorbeeld aan te passen naar een dergelijke architectuur, met behulp van de [officiële Keras-tutorial](https://keras.io/examples/generative/lstm_character_level_text_generation/) als inspiratie.

* **Meerdere lagen LSTM**. Het is zinvol om 2 of 3 lagen van LSTM-cellen te proberen. Zoals we in de vorige eenheid hebben besproken, haalt elke laag van een LSTM bepaalde patronen uit tekst, en in het geval van een generator op tekenniveau kunnen we verwachten dat de lagere LSTM-laag verantwoordelijk is voor het herkennen van lettergrepen, en de hogere lagen - voor woorden en woordcombinaties. Dit kan eenvoudig worden geïmplementeerd door een parameter voor het aantal lagen door te geven aan de LSTM-constructor.

* Je kunt ook experimenteren met **GRU-eenheden** om te zien welke beter presteren, en met **verschillende groottes van verborgen lagen**. Een te grote verborgen laag kan resulteren in overfitting (bijvoorbeeld dat het netwerk exacte tekst leert), terwijl een kleinere grootte mogelijk geen goed resultaat oplevert.


## Zachte tekstgeneratie en temperatuur

In de vorige definitie van `generate` kozen we altijd het karakter met de hoogste waarschijnlijkheid als het volgende karakter in de gegenereerde tekst. Dit resulteerde er vaak in dat de tekst "cyclisch" werd en steeds dezelfde karakterreeksen herhaalde, zoals in dit voorbeeld:
```
today of the second the company and a second the company ...
```

Als we echter naar de waarschijnlijkheidsverdeling voor het volgende karakter kijken, kan het zijn dat het verschil tussen een paar hoogste waarschijnlijkheden niet groot is, bijvoorbeeld: één karakter kan een waarschijnlijkheid van 0,2 hebben, en een ander 0,19, enzovoort. Bijvoorbeeld, bij het zoeken naar het volgende karakter in de reeks '*play*', kan het volgende karakter net zo goed een spatie zijn, of **e** (zoals in het woord *player*).

Dit leidt ons tot de conclusie dat het niet altijd "eerlijk" is om het karakter met de hoogste waarschijnlijkheid te kiezen, omdat het kiezen van het op één na hoogste karakter ook tot betekenisvolle tekst kan leiden. Het is verstandiger om **karakters te bemonsteren** uit de waarschijnlijkheidsverdeling die door de netwerkoutput wordt gegeven.

Dit bemonsteren kan worden gedaan met behulp van de functie `np.multinomial`, die de zogenaamde **multinomiale verdeling** implementeert. Een functie die deze **zachte** tekstgeneratie implementeert, is hieronder gedefinieerd:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

We hebben een extra parameter geïntroduceerd genaamd **temperatuur**, die wordt gebruikt om aan te geven hoe strikt we ons moeten houden aan de hoogste waarschijnlijkheid. Als de temperatuur 1,0 is, doen we eerlijke multinomiale sampling, en wanneer de temperatuur naar oneindig gaat - worden alle waarschijnlijkheden gelijk, en selecteren we willekeurig het volgende teken. In het onderstaande voorbeeld kunnen we zien dat de tekst betekenisloos wordt wanneer we de temperatuur te veel verhogen, en het lijkt op "gecycled" hard gegenereerde tekst wanneer het dichter bij 0 komt.



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, dient u zich ervan bewust te zijn dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in zijn oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor cruciale informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor eventuele misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
